<a href="https://colab.research.google.com/github/sahupra1357/LLMAgents/blob/main/Groq_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [2]:
!pip install groq -q

In [3]:
!pip install pinecone -q

In [4]:
!pip install langchain_community langchain_pinecone -q

In [5]:
#!pip install pandas numpy
#!pip install --upgrade numpy
#!pip install numpy==1.26.4

In [6]:
!pip install --upgrade numpy transformers

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.2.3 requires numpy<2.0.0,>=1.26.4, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.


In [7]:
!pip install tiktoken -q

In [8]:
!pip install chromadb

In [9]:
import pandas as pd
import numpy as np
from groq import Groq
import os
import pinecone

from langchain_community.vectorstores import Chroma
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_pinecone import PineconeVectorStore
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

from IPython.display import display, HTML

In [10]:
client = Groq(api_key = groq_api_key)
model = "llama3-8b-8192"

In [11]:
presidential_speeches_df = pd.read_csv('presidential_speeches.csv')
presidential_speeches_df.head()

,Date,President,Party,Speech Title,Summary,Transcript,URL
0,1789-04-30,George Washington,Unaffiliated,First Inaugural Address,Washington calls on Congress to avoid local an...,Fellow Citizens of the Senate and the House of...,https://millercenter.org/the-presidency/presid...
1,1789-10-03,George Washington,Unaffiliated,Thanksgiving Proclamation,"At the request of Congress, Washington establi...",Whereas it is the duty of all Nations to ackno...,https://millercenter.org/the-presidency/presid...
2,1790-01-08,George Washington,Unaffiliated,First Annual Message to Congress,"In a wide ranging speech, President Washington...",Fellow Citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...
3,1790-12-08,George Washington,Unaffiliated,Second Annual Message to Congress,Washington focuses on commerce in his second a...,Fellow citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...
4,1790-12-29,George Washington,Unaffiliated,Talk to the Chiefs and Counselors of the Senec...,The President reassures the Seneca Nation that...,"I the President of the United States, by my ow...",https://millercenter.org/the-presidency/presid...


In [14]:
presidential_speeches_df.iloc[309].President

'James A. Garfield'

In [15]:
presidential_speeches_df.iloc[307].President

'Rutherford B. Hayes'

In [16]:
presidential_speeches_df.iloc[308].President

'Rutherford B. Hayes'

In [12]:
garfield_inaugural = presidential_speeches_df.iloc[309].Transcript
display(HTML(garfield_inaugural))

In [13]:
model_id = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# create the length function
def token_len(text):
    tokens = tokenizer.encode(
        text
    )
    return len(tokens)

token_len(garfield_inaugural)

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3420 > 512). Running this sequence through the model will result in indexing errors


3420

In [17]:
text_splitter = TokenTextSplitter(
    chunk_size=450, # 500 tokens is the max
    chunk_overlap=20 # Overlap of N tokens between chunks (to reduce chance of cutting out relevant connected text like middle of sentence)
)

chunks = text_splitter.split_text(garfield_inaugural)

for chunk in chunks:
    print(token_len(chunk))

453
455
467
457
457
455
461
368


In [18]:
chunk_embeddings = []
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
for chunk in chunks:
    chunk_embeddings.append(embedding_function.embed_query(chunk))

print(len(chunk_embeddings[0]),chunk_embeddings[0][:20]) #Shows first 25 embeddings out of 384

<ipython-input-18-154714544680>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

384 [-0.04131148383021355, 0.04761349782347679, 0.007975048385560513, -0.030207902193069458, 0.04763733595609665, 0.032533250749111176, 0.012350223958492279, -0.04483683034777641, -0.008013556711375713, 0.01570405624806881, -0.0009442783775739372, 0.11632761359214783, -0.007115584332495928, -0.03356572985649109, -0.04323795437812805, 0.06872354447841644, -0.04552493244409561, -0.07017463445663452, -0.10271696001291275, 0.11116134375333786]


In [19]:
user_question = "What were James Garfield's views on civil service reform?"

In [20]:
prompt_embeddings = embedding_function.embed_query(user_question)
similarities = cosine_similarity([prompt_embeddings], chunk_embeddings)[0]
closest_similarity_index = np.argmax(similarities)
most_relevant_chunk = chunks[closest_similarity_index]
display(HTML(most_relevant_chunk))

In [21]:
# A chat completion function that will use the most relevant exerpt(s) from presidential speeches to answer the user's question
def presidential_speech_chat_completion(client, model, user_question, relevant_excerpts):
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "system",
                "content": "You are a presidential historian. Given the user's question and relevant excerpts from presidential speeches, answer the question by including direct quotes from presidential speeches. When using a quote, site the speech that it was from (ignoring the chunk)."
            },
            {
                "role": "user",
                "content": "User Question: " + user_question + "\n\nRelevant Speech Exerpt(s):\n\n" + relevant_excerpts,
            }
        ],
        model = model
    )

    response = chat_completion.choices[0].message.content
    return response


presidential_speech_chat_completion(client, model, user_question, most_relevant_chunk)

'Based on the provided speech excerpt, James Garfield\'s views on civil service reform are evident. He addresses the issue of patronage and the potential for abuse in the civil service system. Specifically, he states:\n\n"...the civil service can never be placed on a satisfactory basis until it is regulated by law."\n\nThis quote suggests that Garfield believed in the importance of establishing a formal framework for the civil service system to ensure its stability and effectiveness. He also acknowledges the need to protect both appointing officials from undue pressure and incumbents from removals based on political favors.\n\nGarfield also emphasizes the importance of honest and faithful service, stating:\n\n"...it will be the purpose of my Administration to maintain the authority of the nation in all places within its jurisdiction; to enforce obedience to all the laws of the Union in the interests of the people; to demand rigid economy in all the expenditures of the Government, and t

In [22]:
len(presidential_speeches_df.iloc[300:315]['Transcript'])

15

In [39]:
documents = []
ids = []
#for index, row in presidential_speeches_df[presidential_speeches_df['Transcript'].notnull()].iterrows():
for index, row in presidential_speeches_df[300:315].iterrows():
    chunks = text_splitter.split_text(row.Transcript)
    total_chunks = len(chunks)
    for chunk_num in range(1,total_chunks+1):
        header = f"Date: {row['Date']}\nPresident: {row['President']}\nSpeech Title: {row['Speech Title']} (chunk {chunk_num} of {total_chunks})\n\n"
        chunk = chunks[chunk_num-1]
        document_id = f"{index}_chunk_{chunk_num}"
        documents.append(Document(page_content=header + chunk, metadata={"source": "local"}))
        #documents.append(header + chunk)
        ids.append(document_id)
print(len(ids))
print(len(documents))

147
147


In [47]:
len(prompt_embeddings)

384

# PineCone

In [69]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

In [70]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embed.embed_documents(texts)
len(res), len(res[0])

(2, 1536)

In [72]:
from pinecone import Pinecone
pinecone_index_name = 'langchain-retrieval-augmentation'
pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)
dense_index = pc.Index(pinecone_index_name)

In [74]:
# Extract text content from Document objects
document_texts = [doc.page_content for doc in documents]

document_embeddings = embed.embed_documents(document_texts)
vectors = [(doc_id, embedding, {"source": "local"}) for doc_id, embedding in zip(ids, document_embeddings)]

dense_index.upsert(vectors)

{'upserted_count': 147}

In [75]:
stats = dense_index.describe_index_stats()
print(stats)

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 32858}},
 'total_vector_count': 32858}


In [76]:
#pinecone_index_name = "presidential-speeches"
pinecone_docsearch = PineconeVectorStore.from_documents(documents, embed, index_name=pinecone_index_name)

In [77]:
pinecone_user_question = "What were James Garfield's views on civil service reform?"

In [78]:
pinecone_relevent_docs = pinecone_docsearch.similarity_search(pinecone_user_question)

# print results
display(HTML(pinecone_relevent_docs[0].page_content))

In [80]:
pinecone_relevant_excerpts = '\n\n------------------------------------------------------\n\n'.join([doc.page_content for doc in pinecone_relevent_docs[:3]])
display(HTML(pinecone_relevant_excerpts.replace("\n", "<br>")))

In [81]:
presidential_speech_chat_completion(client, model, pinecone_user_question, pinecone_relevant_excerpts)

'James Garfield\'s views on civil service reform are clear from his inaugural address in 1881. He emphasized the importance of regulating the civil service by law, stating:\n\n"The civil service can never be placed on a satisfactory basis until it is regulated by law. For the good of the service itself, for the protection of those who are intrusted with the appointing power against the waste of time and obstruction to the public business caused by the inordinate pressure for place, and for the protection of incumbents against intrigue and wrong, I shall at the proper time ask Congress to fix the tenure of the minor offices of the several Executive Departments and prescribe the grounds upon which removals shall be made during the terms for which incumbents have been appointed."\n\nGarfield also acknowledged the need to prevent the abuse of power by executive officials, saying:\n\n"I shall greatly rely upon the wisdom and patriotism of Congress and of those who may share with me the resp

# Chroma

In [41]:
import chromadb
chroma_index_name = 'presidential-speeches'
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name=chroma_index_name)
document_texts = [doc.page_content for doc in documents]
collection.upsert(
    documents=document_texts,
    ids=ids
)

In [42]:
## Use Chroma for open source option
docsearch = Chroma.from_documents(documents, embedding_function)

In [43]:
user_question = "What were James Garfield's views on civil service reform?"

In [44]:
relevent_docs = docsearch.similarity_search(user_question)

# print results
#display(HTML(relevent_docs[0].page_content))

In [45]:
relevant_excerpts = '\n\n------------------------------------------------------\n\n'.join([doc.page_content for doc in relevent_docs[:3]])
display(HTML(relevant_excerpts.replace("\n", "<br>")))

In [46]:
presidential_speech_chat_completion(client, model, user_question, relevant_excerpts)

'Based on the excerpts from James Garfield\'s Inaugural Address, it can be inferred that Garfield viewed civil service reform as essential for the efficient and fair administration of the government. He explicitly stated that the civil service could "never be placed on a satisfactory basis until it is regulated by law" (Date: 1881-03-04, President: James A. Garfield, Speech Title: Inaugural Address, chunk 8 of 8).\n\nGarfield also emphasized the need for rigid economy in government expenditures and the honest and faithful service of all executive officers. He stressed that the offices were created "not for the benefit of incumbents or their supporters, but for the service of the Government" (Date: 1881-03-04, President: James A. Garfield, Speech Title: Inaugural Address, chunk 8 of 8).\n\nIn another part of the speech, Garfield acknowledged that the Civil War was a "most important political change" that had freed the nation from the perpetual danger of war and dissolution, and had ulti